In [1]:
import Pkg; Pkg.activate(@__DIR__); Pkg.instantiate()

  Activating environment at `/mnt/064AC6424AC62E6D/git_workspace/16715/hopper-sim/scripts/Project.toml`


In [2]:
using RigidBodyDynamics
using LinearAlgebra
using MeshCatMechanisms
using MeshCat
using StaticArrays
using SparseArrays
using ForwardDiff

WebIO._IJuliaInit()

In [3]:
# link lengths
const l0 = [0.1; 0; 0]
const l1 = [0.3; 0; 0]
const l2 = [0.3; 0; 0]
const l3 = 0.1
const l4 = 0.2
const l5 = 0.0205
const lc = [l3; 0; 0]
const lee = [l3 + l4; 0; l5] # sqrt((l3 + l4)^2 + l5^2)

# CoM locations
const l_cb = [0; 0.004; 0]
const l_c0 = [0.0125108364230515; 0.00117191218927888; 0]
const l_c1 = [0.149359714867044; 0; 0]
const l_c2 = [0.0469412900551914; 0; 0]
const l_c3 = [0.113177000131857; 0; -0.015332867880069]

# link masses
const mb = 7  # kg
const m0 = 0.24644240965487
const m1 = 0.0707939028219395
const m2 = 0.276735496985514
const m3 = 0.130824780046739
# const m = Diagonal([m0, m1, m2, m3])
    
# gravity, obviously
const g = 9.807

# mass moment of inertia in axis of rotation
const Ib = Array([0.0024241 5.252E-06 2.0733E-19; 
                  5.252E-06 0.0044176 -3.1153E-19; 
                  2.0733E-19 -3.1153E-19 0.0022481])

const I0 = Array([3.83120149546952E-05 1.46925714738609E-05 -8.60106401672571E-06;
                  1.46925714738609E-05 0.000172067745507247 1.0427260925207E-06;
                  -8.60106401672571E-06 1.0427260925207E-06 0.00014745218068435])

const I1 = Array([3.06999775886187E-06 7.91090301514898E-12 -1.43705963146176E-12;
                  7.91090301514898E-12 0.000147960574744097 1.30742394049546E-11;
                  -1.43705963146176E-12 1.30742394049546E-11 0.000147884231885009])

const I2 = Array([3.43038397803592E-05 -2.90339844227483E-07 6.18680397558952E-06;
                  -2.90339844227483E-07 0.000302324068012293 2.25016327583562E-08;
                  6.18680397558952E-06 2.25016327583562E-08 0.00028292376778719])

const I3 = Array([1.76996970020568E-05 -5.3695427116208E-07 7.62350214406387E-07;
                  -5.3695427116208E-07 0.000164188445564489 -2.77843753828047E-07;
                  7.62350214406387E-07 -2.77843753828047E-07 0.000160656046697151])

3×3 Matrix{Float64}:
  1.76997e-5  -5.36954e-7    7.6235e-7
 -5.36954e-7   0.000164188  -2.77844e-7
  7.6235e-7   -2.77844e-7    0.000160656

In [4]:
M̄ = [mb*I(3) zeros(3, 27)
     zeros(3,3) Ib zeros(3, 24)
     zeros(3,6) m0*I(3) zeros(3,21);
     zeros(3,9) I0 zeros(3,18);
     zeros(3,12) m1*I(3) zeros(3,15);
     zeros(3,15) I1 zeros(3, 12);
     zeros(3,18) m2*I(3) zeros(3, 9);
     zeros(3, 21) I2 zeros(3, 6);
     zeros(3, 24) m3*I(3) zeros(3, 3);
     zeros(3, 27) I3]

30×30 SparseMatrixCSC{Float64, Int64} with 60 stored entries:
⠑⢄⣀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠘⠛⢄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⢱⣶⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠑⢄⣀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠘⠛⢄⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢱⣶⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠑⢄⣀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠘⠛

In [5]:
function hat(ω)
    return [0 -ω[3] ω[2];
            ω[3] 0 -ω[1];
            -ω[2] ω[1] 0]
end

function L(Q)
    [Q[1] -Q[2:4]'; Q[2:4] Q[1]*I + hat(Q[2:4])]
end

function R(Q)
    [Q[1] -Q[2:4]'; Q[2:4] Q[1]*I - hat(Q[2:4])]
end

H = [zeros(1,3); I];

T = Diagonal([1.0; -1; -1; -1])

function G(Q)
    return L(Q)*H  # 4x3
end

function Ḡ(q)
    Qb = q[4:7]
    Q0 = q[11:14]
    Q1 = q[18:21]
    Q2 = q[25:28]
    Q3 = q[32:35]
    
    # [I(3) zeros(3,4); zeros(4,3) G(Q) zeros(4,1); zeros(1,6) 1]  # 8x7
    return blockdiag(sparse(I, 3, 3), sparse(G(Qb)), 
                     sparse(I, 3, 3), sparse(G(Q0)), 
                     sparse(I, 3, 3), sparse(G(Q1)),
                     sparse(I, 3, 3), sparse(G(Q2)),
                     sparse(I, 3, 3), sparse(G(Q3)),)
end

Ḡ (generic function with 1 method)

In [6]:
function Expq(ϕ)
    
    # The quaternion exponential map ϕ → q 
    q = zeros(4)
    θ = norm(ϕ)
    q = [cos(θ/2); 0.5*ϕ*sinc(θ/(2*pi))]
    
    return q
end

function rotate(Q, p)
    # Rotate a position vector p by a quaternion Q
    return H'L(Q)*R(Q)'*H*p
end

rotate (generic function with 1 method)

In [7]:
#Input Jacobian
function B(q)
    #[zeros(7); 1.0; zeros(2)]
    I(10)
end

B (generic function with 1 method)

In [8]:
function del(m, I, r1, r2, r3, Q1, Q2, Q3)
    [(1/h)*m*(r2-r1) - (1/h)*m*(r3-r2);
     (2.0/h)*G(Q2)'*L(Q1)*H*I*H'*L(Q1)'*Q2 + (2.0/h)*G(Q2)'*T*R(Q3)'*H*I*H'*L(Q2)'*Q3]
end
    
    
function DEL(q_1,q_2,q_3,λ,F1,F2)
    
    rb_1 = q_1[1:3]
    Qb_1 = q_1[4:7]
    r0_1 = q_1[8:10]
    Q0_1 = q_1[11:14]
    r1_1 = q_1[15:17]
    Q1_1 = q_1[18:21]
    r2_1 = q_1[22:24]
    Q2_1 = q_1[25:28]
    r3_1 = q_1[29:31]
    Q3_1 = q_1[32:35]
    
    rb_2 = q_2[1:3]
    Qb_2 = q_2[4:7]
    r0_2 = q_2[8:10]
    Q0_2 = q_2[11:14]
    r1_2 = q_2[15:17]
    Q1_2 = q_2[18:21]
    r2_2 = q_2[22:24]
    Q2_2 = q_2[25:28]
    r3_2 = q_2[29:31]
    Q3_2 = q_2[32:35]
    
    rb_3 = q_3[1:3]
    Qb_3 = q_3[4:7]
    r0_3 = q_3[8:10]
    Q0_3 = q_3[11:14]
    r1_3 = q_3[15:17]
    Q1_3 = q_3[18:21]
    r2_3 = q_3[22:24]
    Q2_3 = q_3[25:28]
    r3_3 = q_3[29:31]
    Q3_3 = q_3[32:35]

    del1 = [del(mb, Ib, rb_1, rb_2, rb_3, Qb_1, Qb_2, Qb_3);
            del(m0, I0, r0_1, r0_2, r0_3, Q0_1, Q0_2, Q0_3);
            del(m1, I1, r1_1, r1_2, r1_3, Q1_1, Q1_2, Q1_3);
            del(m2, I2, r2_1, r2_2, r2_3, Q2_1, Q2_2, Q2_3);
            del(m3, I3, r3_1, r3_2, r3_3, Q3_1, Q3_2, Q3_3)] 

    return del1 + (h/2.0)*F1 + (h/2.0)*F2 + h*Dc(q_2)'*λ
end

DEL (generic function with 1 method)

In [9]:
function Dq3DEL(q_1,q_2,q_3,λ,F1,F2)
    # @show Ḡ(q_3)
    ForwardDiff.jacobian(dq->DEL(q_1,q_2,dq,λ,F1,F2), q_3)*Ḡ(q_3)
end

Dq3DEL (generic function with 1 method)

In [10]:
function c(q)
    # constraint function
    # c(q_0) should be all zeros
    rb = q[1:3]
    Qb = q[4:7]
    r0 = q[8:10]
    Q0 = q[11:14]
    r1 = q[15:17]
    Q1 = q[18:21]
    r2 = q[22:24]
    Q2 = q[25:28]
    r3 = q[29:31]
    Q3 = q[32:35]
    
    pb = rb + rotate(Qb, l_cb) # position vector from world frame to *JOINTS* 0 and 2
    
    c_ = [pb - r0 - rotate(Q0, -l_c0);  # ???
         [0 1 0 0; 0 0 0 1]*L(Qb)'*Q0;  # y axis rotation constraint
          r0 + rotate(Q0,  l0-l_c0) - r1 - rotate(Q1, -l_c1);
         [0 1 0 0; 0 0 0 1]*L(Q0)'*Q1;
          pb - r2 - rotate(Q2, -l_c2);
         [0 1 0 0; 0 0 0 1]*L(Qb)'*Q2;
          r2 + rotate(Q2, l2 - l_c2) - r3 - rotate(Q3, -l_c3);
         [0 1 0 0; 0 0 0 1]*L(Q2)'*Q3;
          r1 + rotate(Q1, l1 - l_c1) - r3 - rotate(Q3, lc-l_c3)]
         #[0 1 0 0; 0 0 0 1]*L(Q1)'*Q3]
end

c (generic function with 1 method)

In [11]:
function Dc(q)
    ForwardDiff.jacobian(dq->c(dq),q)*Ḡ(q)
end

Dc (generic function with 1 method)

In [21]:
#Initial Conditions
rb = zeros(3)
Qb = [1.0; 0; 0; 0]
q0 = 30*pi/180;
q1 = 120*(pi/180)
q2 = 150*(pi/180)
q3 = -120*(pi/180)

h = 0.001 #20 Hz
Tf = 10.0
thist = 0:h:Tf
N = length(thist)

pb = rb + rotate(Qb, l_cb)  # position vector from world frame to *JOINTS* 0 and 2

Qb0 = Expq([0, q0, 0])  # quaternion from base to link 0
Q0 = L(Qb)*Qb0  # quaternion from world frame to link 0
r0 = pb + rotate(Q0, l_c0)

Q01 = Expq([0, q1, 0])  # quaternion from link 0 to link 1
Q1 = L(Q0)*Q01  # quaternion from world frame to link 1
r1 = pb + rotate(Q0, l0) + rotate(Q1, l_c1)  

Qb2 = Expq([0, q2, 0])  # quaternion from base to link 2
Q2 = L(Qb)*Qb2  # quaternion from world frame to link 2
r2 = pb + rotate(Q2, l_c2)

Q23 = Expq([0, q3, 0])  # quaternion from base to link 2
Q3 = L(Q2)*Q23  # quaternion from world frame to link 2
r3 = pb + rotate(Q2, l2) + rotate(Q3, l_c3)  

rb_0 = rb
r0_0 = r0
r1_0 = r1
r2_0 = r2
r3_0 = r3
Qb_0 = Qb
Q0_0 = Q0
Q1_0 = Q1
Q2_0 = Q2
Q3_0 = Q3

q_0 = [rb_0; Qb_0; r0_0; Q0_0; r1_0; Q1_0; r2_0; Q2_0; r3_0; Q3_0]
# @show c(q_0)


35-element Vector{Float64}:
  0.0
  0.0
  0.0
  1.0
  0.0
  0.0
  0.0
  0.010834702164954238
  0.00517191218927888
 -0.006255418211525749
  0.9659258262890683
  0.0
  0.25881904510252074
  ⋮
 -0.023470645027595695
  0.25881904510252074
  0.0
  0.9659258262890683
  0.0
 -0.16945989783706322
  0.004
 -0.2198671531629387
  0.9659258262890683
  0.0
  0.2588190451025209
  0.0

In [22]:
#Torque input at joints
uhist = repeat([1 0 0 0 0]*1e-4, N)'
@show size(uhist)
# 1 -> joint0
# 2 -> joint1
# 3 -> joint2
# 4 -> joint3
# 5 -> joint4 (parallel constraint)
#Corresponding F
Fhist = zeros(30,N)  # wrench [xyz force, xyz torque]
for k = 1:N
    Fhist[:,k] = [zeros(4); -uhist[1, k]-uhist[3, k]; 0; # body
                  zeros(4); uhist[1, k]-uhist[2, k]; 0; # link0
                  zeros(4); uhist[2, k]+uhist[5, k]; 0; # link1
                  zeros(4); uhist[3, k]-uhist[4, k]; 0; # link2
                  zeros(4); uhist[4, k]-uhist[5, k]; 0] # link3
end

size(uhist) = (5, 10001)


In [23]:
#Simulate
qhist = zeros(35,N)
qhist[:,1] .= q_0
qhist[:,2] .= q_0

@show n_c = size(c(q_0))[1]  # number of constraint rows

for k = 2:(N-1)
    
    #Initial guess
    qhist[:,k+1] .= qhist[:,k]
    λ = zeros(n_c)
    
    e = [DEL(qhist[:,k-1],qhist[:,k],qhist[:,k+1],λ,Fhist[:,k-1],Fhist[:,k]); c(qhist[:,k+1])]

    while maximum(abs.(e)) > 1e-12
        D = Dq3DEL(qhist[:,k-1],qhist[:,k],qhist[:,k+1],λ,Fhist[:,k-1],Fhist[:,k])
        C2 = Dc(qhist[:,k])
        C3 = Dc(qhist[:,k+1])

        Δ = -([D h*C2'; C3 zeros(n_c,n_c)]\e)
        
        qhist[1:3,k+1] .= qhist[1:3,k+1] + Δ[1:3]
        qhist[4:7,k+1] .= L(qhist[4:7,k+1])*[sqrt(1-Δ[4:6]'*Δ[4:6]); Δ[4:6]]
        qhist[8:10,k+1] .= qhist[8:10,k+1] + Δ[7:9]
        qhist[11:14,k+1] .= L(qhist[11:14,k+1])*[sqrt(1-Δ[10:12]'*Δ[10:12]); Δ[10:12]]
        qhist[15:17,k+1] .= qhist[15:17,k+1] + Δ[13:15]
        qhist[18:21,k+1] .= L(qhist[18:21,k+1])*[sqrt(1-Δ[16:18]'*Δ[16:18]); Δ[16:18]]
        qhist[22:24,k+1] .= qhist[22:24,k+1] + Δ[19:21]
        qhist[25:28,k+1] .= L(qhist[25:28,k+1])*[sqrt(1-Δ[22:24]'*Δ[22:24]); Δ[22:24]]
        qhist[29:31,k+1] .= qhist[29:31,k+1] + Δ[25:27]
        qhist[32:35,k+1] .= L(qhist[32:35,k+1])*[sqrt(1-Δ[28:30]'*Δ[28:30]); Δ[28:30]]

        λ .= λ + Δ[31:30+n_c]
        
        e = [DEL(qhist[:,k-1],qhist[:,k],qhist[:,k+1],λ,Fhist[:,k-1],Fhist[:,k]); c(qhist[:,k+1])]
        # print(maximum(abs.(e)), "\n") # 
        # print(c(qhist[:,k+1]), "\n")
        # flush(stdout)
    end
    print("\r Simulation ", trunc(Int, k/(N-1)*100), "% complete")
    flush(stdout)
end

n_c = (size(c(q_0)))[1] = 23
 Simulation 100% completecomplete% complete% complete% complete

In [24]:
qhist

35×10001 Matrix{Float64}:
  0.0          0.0          5.60678e-11  …   0.00283294   0.00283349
  0.0          0.0         -9.50487e-13     -5.95019e-5  -5.95152e-5
  0.0          0.0          3.5572e-11       0.00168979   0.00169011
  1.0          1.0          1.0              0.844162     0.844101
  0.0          0.0         -6.42461e-11     -0.00691312  -0.00691486
  0.0          0.0         -1.13183e-8   …  -0.536024    -0.536119
  0.0          0.0         -1.95483e-10     -0.00466298  -0.00466284
  0.0108347    0.0108347    0.0108347        0.00254124   0.00253863
  0.00517191   0.00517191   0.00517191       0.0050339    0.00503392
 -0.00625542  -0.00625542  -0.00625542      -0.0108498   -0.0108494
  0.965926     0.965926     0.965926     …   0.696544     0.696452
  0.0          0.0         -1.14623e-11      0.00315913   0.0031605
  0.258819     0.258819     0.258819         0.717466     0.717554
  ⋮                                      ⋱                ⋮
 -0.0234706   -0.0234706   

In [16]:
using MeshCat, GeometryBasics, CoordinateTransformations, Rotations#, GeometryTypes
#const gt = GeometryTypes

In [17]:
vis = Visualizer()
render(vis)

┌ Info: MeshCat server started. You can open the visualizer by visiting the following URL in your browser:
│ http://127.0.0.1:8704
└ @ MeshCat /home/ben/.julia/packages/MeshCat/GlCMx/src/visualizer.jl:73


MeshCat.DisplayedVisualizer(MeshCat.CoreVisualizer(MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()), Set{Any}(), ip"127.0.0.1", 8704))

In [27]:
import Plots as pl

function qinv(Q)
    return reshape(reshape(Q, 1, 4)*Diagonal(T)/(norm(Q)^2), 4)
end

delete!(vis)

green_material = MeshPhongMaterial(color=pl.RGBA(0, 1, 0, 0.8))
red_material = MeshPhongMaterial(color=pl.RGBA(1, 0, 0, 0.8))

#l_345 = sqrt((l3 + l4)^2 + l5^2)
# sphere = Sphere(Point(0, 0, 0), 3)
# rect = Rectangle(0.2, 0.2, 0.2, 0.2)
cylinderb = Cylinder(Point(0, -0.02, l_cb[2]), Point(0, 0.02, l_cb[2]), 0.04)  #float(l_cb)
cylinder0 = Cylinder(Point(-l_c0[1], -l_c0[2], -l_c0[3]), Point((l0-l_c0)[1], (l0-l_c0)[2], (l0-l_c0)[3]), 0.008)
cylinder1 = Cylinder(Point(-l_c1[1], -l_c1[2], -l_c1[3]), Point((l1-l_c1)[1],(l1-l_c1)[2],(l1-l_c1)[2]), 0.008)
cylinder2 = Cylinder(Point(-l_c2[1], -l_c2[2], -l_c2[3]), Point((l2-l_c2)[1], (l2-l_c2)[2], (l2-l_c2)[3]), 0.008)
cylinder3 = Cylinder(Point(-l_c3[1], -l_c3[2], -l_c3[3]), Point((lee-l_c3)[1], (lee-l_c3)[2],  (lee-l_c3)[3]), 0.008)

# setobject!(vis["sphere"],sphere,red_material)
setobject!(vis["cylinderb"],cylinderb,red_material)
setobject!(vis["cylinder0"],cylinder0,green_material)
setobject!(vis["cylinder1"],cylinder1,green_material)
setobject!(vis["cylinder2"],cylinder2,green_material)
setobject!(vis["cylinder3"],cylinder3,green_material)

for k = 1:N
    bl = Diagonal([1, 1, -1])
    q = qhist[:, k]
    # set position and attitude
    
    positionb = Translation(q[1:3]...)
    attitudeb = LinearMap(UnitQuaternion(q[4:7]))
    position0 = Translation(q[8:10]...)
    attitude0 = LinearMap(UnitQuaternion(q[11:14]))
    position1 = Translation(q[15:17]...)
    attitude1 = LinearMap(UnitQuaternion(q[18:21]))
    position2 = Translation(q[22:24]...)
    attitude2 = LinearMap(UnitQuaternion(q[25:28]))
    position3 = Translation(q[29:31]...)
    attitude3 = LinearMap(UnitQuaternion(q[32:35]))
    
    #=
    positionb = Translation(reshape(q[1:3], 1, 3)*bl...)
    attitudeb = LinearMap(UnitQuaternion(qinv(q[4:7])))
    position0 = Translation(reshape(q[8:10], 1, 3)*bl...)
    attitude0 = LinearMap(UnitQuaternion(qinv(q[11:14])))
    position1 = Translation(reshape(q[15:17], 1, 3)*bl...)
    attitude1 = LinearMap(UnitQuaternion(qinv(q[18:21])))
    position2 = Translation(reshape(q[22:24], 1, 3)*bl...)
    attitude2 = LinearMap(UnitQuaternion(qinv(q[25:28])))
    position3 = Translation(reshape(q[29:31], 1, 3)*bl...)
    attitude3 = LinearMap(UnitQuaternion(qinv(q[32:35])))
    =#
    settransform!(vis["cylinderb"], compose(positionb,attitudeb))
    settransform!(vis["cylinder0"], compose(position0,attitude0))
    settransform!(vis["cylinder1"], compose(position1,attitude1))
    settransform!(vis["cylinder2"], compose(position2,attitude2))
    settransform!(vis["cylinder3"], compose(position3,attitude3))
    #sleep(0.001)
end

In [19]:
# only works with Rotations v1.0.4 and earlier
# use Pkg.add(Pkg.PackageSpec(;name="Rotations", version="1.0.4"))
curdir = pwd()
urdfpath = joinpath(curdir, "../res/flyhopper_robot/urdf/flyhopper_robot.urdf")
leg = parse_urdf(urdfpath, floating=true)
state = MechanismState(leg)
# mvis = MechanismVisualizer(doublependulum, Skeleton(randomize_colors=true, inertias=false));
mvis = MechanismVisualizer(leg, URDFVisuals(urdfpath));

render(mvis)

┌ Info: MeshCat server started. You can open the visualizer by visiting the following URL in your browser:
│ http://127.0.0.1:8705
└ @ MeshCat /home/ben/.julia/packages/MeshCat/GlCMx/src/visualizer.jl:73


MeshCat.DisplayedVisualizer(MeshCat.CoreVisualizer(MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("meshcat" => MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("world" => MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("base_link" => MeshCat.SceneTrees.SceneNode(nothing, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x2f, 0x62, 0x61, 0x73, 0x65, 0x5f, 0x6c, 0x69, 0x6e, 0x6b], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("base_link" => MeshCat.SceneTrees.SceneNode(nothing, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x2f, 0x62, 0x61, 0x73, 0x65, 0x5f, 0x6c, 0x69, 0x6e, 0x6b], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("geometry_1" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x67, 0x65, 0x6f, 0x6d, 0x65, 0x74, 0x72, 0x79, 0x5f, 0x31], UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x67, 0x65, 0x6f, 0x6d, 0x65, 0x74, 0x72, 0x79, 0x5f, 0x31], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()))), "Link 2" => MeshCat.SceneTrees.SceneNode(nothing, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x69, 0x6e, 0x6b, 0x2f, 0x4c, 0x69, 0x6e, 0x6b, 0x20, 0x32], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("Link 3" => MeshCat.SceneTrees.SceneNode(nothing, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x6b, 0x20, 0x32, 0x2f, 0x4c, 0x69, 0x6e, 0x6b, 0x20, 0x33], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("after_Joint 3" => MeshCat.SceneTrees.SceneNode(nothing, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x65, 0x72, 0x5f, 0x4a, 0x6f, 0x69, 0x6e, 0x74, 0x20, 0x33], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("geometry_5" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x67, 0x65, 0x6f, 0x6d, 0x65, 0x74, 0x72, 0x79, 0x5f, 0x35], UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x67, 0x65, 0x6f, 0x6d, 0x65, 0x74, 0x72, 0x79, 0x5f, 0x35], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()))))), "after_Joint 2" => MeshCat.SceneTrees.SceneNode(nothing, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x65, 0x72, 0x5f, 0x4a, 0x6f, 0x69, 0x6e, 0x74, 0x20, 0x32], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("geometry_4" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x67, 0x65, 0x6f, 0x6d, 0x65, 0x74, 0x72, 0x79, 0x5f, 0x34], UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x67, 0x65, 0x6f, 0x6d, 0x65, 0x74, 0x72, 0x79, 0x5f, 0x34], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()))))), "Link 0" => MeshCat.SceneTrees.SceneNode(nothing, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x69, 0x6e, 0x6b, 0x2f, 0x4c, 0x69, 0x6e, 0x6b, 0x20, 0x30], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("Link 1" => MeshCat.SceneTrees.SceneNode(nothing, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x6b, 0x20, 0x30, 0x2f, 0x4c, 0x69, 0x6e, 0x6b, 0x20, 0x31], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("after_Joint 1" => MeshCat.SceneTrees.SceneNode(nothing, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x65, 0x72, 0x5f, 0x4a, 0x6f, 0x69, 0x6e, 0x74, 0x20, 0x31], Dict{String, Vector{UInt8}}(), nothing

In [28]:
function qinv(Q)
    # quaternion inverse = quaternion conjugate
    return reshape(Q, 1, 4)*Diagonal(T)/(norm(Q)^2)
end

function anglesolve(Q)
    2 * atan(norm(Q[2:4]),Q[1])
end

for k = 1:N
    
    q = copy(qhist[:, k])
    pb = q[1:3]
    Qb = q[4:7]
    Q0 = q[11:14]
    Q1 = q[18:21]
    Q2 = q[25:28]
    Q3 = q[32:35]
    
    Qb0 = L(Qb)'*Q0
    Q01 = L(Q0)'*Q1
    Qb2 = L(Qb)'*Q2
    Q23 = L(Q2)'*Q3

    # convert quaternions to joint angles
    a0 = -anglesolve(Qb0) +30*(pi/180)
    a1 = -anglesolve(Q01) +120*(pi/180)
    a2 = -anglesolve(Qb2) +150*(pi/180)
    a3 = anglesolve(Q23) -120*(pi/180)

    #-0.5235987755982988, -2.6179938779914944, -2.0943951023931953, 2.0943951023931953
    
    q_array = vcat(Qb, pb, [a0, a2, a1, a3])
    @show [a0, a2, a1, a3]
    set_configuration!(mvis, q_array)
    
end

[a0, a2, a1, a3] = [0.0, 0.0, 0.0, 0.0]
[a0, a2, a1, a3] = [0.0, 0.0, 0.0, 0.0]
[a0, a2, a1, a3] = [-4.532455388783774e-8, -2.619318983931862e-8, 1.9131363604429907e-8, -1.9131364048519117e-8]
[a0, a2, a1, a3] = [-1.359736605532902e-7, -7.857956951795586e-8, 5.739409081328972e-8, -5.739409081328972e-8]
[a0, a2, a1, a3] = [-2.719473188861343e-7, -1.5715913681546567e-7, 1.1478818162657944e-7, -1.1478818207066865e-7]
[a0, a2, a1, a3] = [-4.532455276651248e-7, -2.6193189084366963e-7, 1.9131363648838828e-7, -1.913136369324775e-7]
[a0, a2, a1, a3] = [-6.798682841147041e-7, -3.928978284939433e-7, 2.8697045495462703e-7, -2.8697045539871624e-7]
[a0, a2, a1, a3] = [-9.518155847931808e-7, -5.500569471017513e-7, 4.017586374693849e-7, -4.017586388016525e-7]
[a0, a2, a1, a3] = [-1.269087427480109e-6, -7.334092431143802e-7, 5.356781844767511e-7, -5.356781853649295e-7]
[a0, a2, a1, a3] = [-1.631683806513351e-6, -9.429547116468484e-7, 6.887290950885472e-7, -6.887290964208148e-7]
[a0, a2, a1, a3] = [-2.

[a0, a2, a1, a3] = [-0.00018972682541373853, -0.00010964289428017082, 8.008393113323464e-5, -8.008393113412282e-5]
[a0, a2, a1, a3] = [-0.00019389660629587357, -0.00011205258779822813, 8.184401849709033e-5, -8.184401849797851e-5]
[a0, a2, a1, a3] = [-0.00019811170915839948, -0.00011448847187089939, 8.362323728716703e-5, -8.36232372884993e-5]
[a0, a2, a1, a3] = [-0.00020237213394613818, -0.00011695054643956482, 8.542158750657336e-5, -8.542158750790563e-5]
[a0, a2, a1, a3] = [-0.0002066778806021352, -0.00011943881144693691, 8.723906915486523e-5, -8.72390691561975e-5]
[a0, a2, a1, a3] = [-0.000211028949068659, -0.00012195326683483998, 8.907568223337492e-5, -8.907568223470719e-5]
[a0, a2, a1, a3] = [-0.00021542533928808894, -0.00012449391254332198, 9.093142674432286e-5, -9.093142674565513e-5]
[a0, a2, a1, a3] = [-0.00021986705120269345, -0.00012706074851331906, 9.280630268904133e-5, -9.28063026903736e-5]
[a0, a2, a1, a3] = [-0.00022435408475329766, -0.00012965377468399097, 9.47003100693066

[a0, a2, a1, a3] = [-0.0007301525467612313, -0.000421945639688559, 0.0003082069070727833, -0.000308206907076336]
[a0, a2, a1, a3] = [-0.000738310381940499, -0.0004266598150124423, 0.00031165056692872284, -0.00031165056693227555]
[a0, a2, a1, a3] = [-0.0007465135318804883, -0.0004314001734897488, 0.0003151133583911836, -0.0003151133583942922]
[a0, a2, a1, a3] = [-0.0007547619964722863, -0.00043616671500990023, 0.0003185952814628301, -0.00031859528146638283]
[a0, a2, a1, a3] = [-0.0007630557756059808, -0.00044095943946009797, 0.00032209633614632693, -0.00032209633615032374]
[a0, a2, a1, a3] = [-0.0007713948691739914, -0.00044577834672843153, 0.00032561652244611494, -0.00032561652244922357]
[a0, a2, a1, a3] = [-0.0007797792770629641, -0.0004506234367012141, 0.00032915584036175005, -0.00032915584036530277]
[a0, a2, a1, a3] = [-0.000788208999162876, -0.00045549470926475877, 0.00033271428989856133, -0.0003327142899030022]
[a0, a2, a1, a3] = [-0.0007966840353619276, -0.0004603921643036024, 0.

[a0, a2, a1, a3] = [-0.0016336386653349466, -0.0009440263038409391, 0.0006896123614947847, -0.0006896123614992256]
[a0, a2, a1, a3] = [-0.0016458290204149906, -0.0009510702733375176, 0.0006947587470782501, -0.0006947587470818029]
[a0, a2, a1, a3] = [-0.0016580646782362596, -0.0009581404136573823, 0.0006999242645795434, -0.0006999242645835402]
[a0, a2, a1, a3] = [-0.0016703456386368831, -0.0009652367246331117, 0.0007051089140044375, -0.0007051089140084343]
[a0, a2, a1, a3] = [-0.0016826719014542135, -0.0009723592060981723, 0.0007103126953569294, -0.0007103126953609262]
[a0, a2, a1, a3] = [-0.001695043466525048, -0.0009795078578851424, 0.0007155356086405718, -0.0007155356086441245]
[a0, a2, a1, a3] = [-0.0017074603336847405, -0.0009866826798261563, 0.0007207776538598054, -0.0007207776538633581]
[a0, a2, a1, a3] = [-0.0017199225027707543, -0.000993883671752016, 0.0007260388310195154, -0.0007260388310235122]
[a0, a2, a1, a3] = [-0.0017324299736171112, -0.0010011108334939678, 0.000731319140

[a0, a2, a1, a3] = [-0.00291218755756395, -0.0016827766097029517, 0.0012294109478623305, -0.0012294109478712123]
[a0, a2, a1, a3] = [-0.0029284544409655666, -0.0016921752155281133, 0.0012362792254396737, -0.0012362792254481114]
[a0, a2, a1, a3] = [-0.0029447666101549697, -0.0017015999747611588, 0.0012431666353953652, -0.001243166635403803]
[a0, a2, a1, a3] = [-0.0029611240649153325, -0.0017110508871813757, 0.0012500731777356222, -0.00125007317774406]
[a0, a2, a1, a3] = [-0.00297752680503216, -0.0017205279525662753, 0.0012569988524675502, -0.001256998852476432]
[a0, a2, a1, a3] = [-0.002993974830288737, -0.0017300311706938132, 0.0012639436595964781, -0.0012639436596049158]
[a0, a2, a1, a3] = [-0.0030104681404671263, -0.0017395605413406123, 0.0012709075991281793, -0.001270907599136617]
[a0, a2, a1, a3] = [-0.0030270067353509456, -0.0017491160642824077, 0.001277890671069759, -0.0012778906710773086]
[a0, a2, a1, a3] = [-0.0030435906147210368, -0.0017586977392958225, 0.0012848928754265465, 

...

[a0, a2, a1, a3] = [-0.45693925458584406, -0.2590033228723656, 0.19793593171347257, -0.1979359317134617]
[a0, a2, a1, a3] = [-0.4571376922224678, -0.2591133660740228, 0.19802432614843868, -0.19802432614842846]
[a0, a2, a1, a3] = [-0.4573361714412748, -0.2592234300014611, 0.19811274143980717, -0.1981127414397963]
[a0, a2, a1, a3] = [-0.4575346922415936, -0.2593335146525386, 0.19820117758904865, -0.1982011775890382]
[a0, a2, a1, a3] = [-0.45773325462275316, -0.2594436200251109, 0.19828963459763616, -0.19828963459762572]
[a0, a2, a1, a3] = [-0.45793185858408636, -0.25955374611703563, 0.19837811246704473, -0.1983781124670343]
[a0, a2, a1, a3] = [-0.45813050412492584, -0.2596638929261683, 0.19846661119875142, -0.19846661119874032]
[a0, a2, a1, a3] = [-0.4583291912446028, -0.25977406045036533, 0.19855513079423104, -0.19855513079421994]
[a0, a2, a1, a3] = [-0.45852791994244935, -0.2598842486874817, 0.1986436712549613, -0.19864367125494953]
[a0, a2, a1, a3] = [-0.4587266902177999, -0.259994457

[a0, a2, a1, a3] = [-0.47658368357633185, -0.2698857391811771, 0.20669794439514844, -0.20669794439513733]
[a0, a2, a1, a3] = [-0.4767861931839573, -0.2699978032748569, 0.20678838990909432, -0.20678838990908344]
[a0, a2, a1, a3] = [-0.47698874431131577, -0.270109887882763, 0.20687885642854664, -0.20687885642853598]
[a0, a2, a1, a3] = [-0.4771913369578059, -0.27022199300272076, 0.20696934395507904, -0.20696934395506816]
[a0, a2, a1, a3] = [-0.47739397112282667, -0.27033411863255496, 0.2070598524902656, -0.20705985249025427]
[a0, a2, a1, a3] = [-0.47759664680577385, -0.2704462647700887, 0.20715038203567904, -0.2071503820356677]
[a0, a2, a1, a3] = [-0.4777993640060506, -0.2705584314131473, 0.20724093259289678, -0.20724093259288479]
[a0, a2, a1, a3] = [-0.4780021227230532, -0.27067061855955377, 0.20733150416349355, -0.20733150416348178]
[a0, a2, a1, a3] = [-0.47820492295618544, -0.2707828262071308, 0.20742209674904855, -0.20742209674903678]
[a0, a2, a1, a3] = [-0.4784077647048478, -0.270895

[a0, a2, a1, a3] = [-0.4986945030850588, -0.28210674666509394, 0.2165877564199592, -0.21658775641994854]
[a0, a2, a1, a3] = [-0.4989014934908932, -0.2822210136598069, 0.21668047983108063, -0.21668047983106975]
[a0, a2, a1, a3] = [-0.4991085253556208, -0.2823353009319156, 0.21677322442369995, -0.21677322442368907]
[a0, a2, a1, a3] = [-0.49931559867872133, -0.2824496084792094, 0.21686599019950648, -0.21686599019949537]
[a0, a2, a1, a3] = [-0.49952271345967214, -0.2825639362994763, 0.21695877716018974, -0.21695877716017886]
[a0, a2, a1, a3] = [-0.4997298696979532, -0.28267828439050513, 0.2170515853074424, -0.21705158530743196]
[a0, a2, a1, a3] = [-0.49993706739304444, -0.28279265275008303, 0.21714441464295553, -0.21714441464294532]
[a0, a2, a1, a3] = [-0.5001443065444265, -0.2829070413759984, 0.21723726516842223, -0.21723726516841158]
[a0, a2, a1, a3] = [-0.5003515871515812, -0.283021450266038, 0.21733013688553693, -0.2173301368855265]
[a0, a2, a1, a3] = [-0.50055890921399, -0.28313587941

Excessive output truncated after 524385 bytes.


[a0, a2, a1, a3] = [-0.5193875914014762, -0.29351719671259824, 0.22587039468887204, -0.22587039468886472]
[a0, a2, a1, a3] = [-0.5195986838280159, -0.293633460390581, 0.22596522343742853, -0.22596522343742076]
[a0, a2, a1, a3] = [-0.5198098176655906, -0.2937497441254231, 0.22606007354016233, -0.226060073540155]
[a0, a2, a1, a3] = [-0.5200209929137588, -0.29386604791487914, 0.22615494499887445, -0.22615494499886646]
[a0, a2, a1, a3] = [-0.5202322095720767, -0.2939823717567047, 0.22624983781536656, -0.22624983781535946]
[a0, a2, a1, a3] = [-0.5204434676401052, -0.29409871564865453, 0.226344751991445, -0.22634475199143744]
[a0, a2, a1, a3] = [-0.5206547671174021, -0.29421507958848236, 0.22643968752891408, -0.22643968752890653]
[a0, a2, a1, a3] = [-0.5208661080035287, -0.29433146357394335, 0.2265346444295795, -0.22653464442957238]
[a0, a2, a1, a3] = [-0.5210774902980454, -0.29444786760278996, 0.22662962269524978, -0.2266296226952429]
[a0, a2, a1, a3] = [-0.5212889140005156, -0.29456429167